### Загрузка всех необходимых *библиотек* и *пакетов*

In [1]:
import os
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import math
from tqdm import tqdm
import torch.nn.functional as F

2024-05-11 14:08:41.160965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 14:08:45.101298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Загрузка и первичный анализ данных

#### Загрузим данные и проверим их на наличие пропущенных значений:

In [2]:
# Загрузка данных
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')

# Проверка на наличие пропущенных значений
print("Пропущенные значения в обучающем наборе:", train_data.isnull().sum().sum())
print("Пропущенные значения в тестовом наборе:", test_data.isnull().sum().sum())


Пропущенные значения в обучающем наборе: 0
Пропущенные значения в тестовом наборе: 0


#### Визуализация данных


In [3]:
def plot_images(data, row, col):
    fig, axes = plt.subplots(row, col, figsize=(12, 12))
    axes = axes.ravel()
    for i in range(row * col):
        img = data.iloc[i, 1:].values.reshape(28, 28)
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(f'Label: {data.iloc[i, 0]}', fontsize=12)
        axes[i].axis('off')
    plt.subplots_adjust(wspace=0.5)

# Визуализация первых 10 изображений
# plot_images(train_data, 2, 5)


### Предобработка данных

In [4]:
def data_preprocessing(raw):
    labels = torch.tensor(raw.iloc[:, 0].values).long()
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = torch.tensor(x_as_array).view(num_images, 1, 28, 28).float()
    x_shaped_array /= 255.0
    return x_shaped_array, labels

X, y = data_preprocessing(train_data)
X_test, y_test = data_preprocessing(test_data)

# Разделение обучающих данных на обучающий и проверочный набор с соотношением разделения 75-25 %
dataset = TensorDataset(X, y)
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [5]:
# Загрузчики данных
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=128, shuffle=False)

### Сверточная нейронная сеть

#### Задаем модель в **PyTorch**

In [6]:
# Модель
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Первый сверточный слой
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding='same')
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Второй сверточный слой
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding='same')
        self.pool2 = nn.MaxPool2d(2, 2)

        # Дропаут и BatchNorm
        self.dropout1 = nn.Dropout(0.3)
        self.bn1 = nn.BatchNorm2d(64)
        
        # Третий и четвертый сверточные слои
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding='same')
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding='same')
        self.pool3 = nn.MaxPool2d(2, 2)

        # Дропаут
        self.dropout2 = nn.Dropout(0.4)

        # Полносвязные слои
        self.flatten = nn.Flatten()
        self.bn2 = nn.BatchNorm1d(1152)
        self.fc1 = nn.Linear(1152, 512)  # Указан размер входа согласно размерам после пулинга
        self.dropout3 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = self.dropout1(x)
        x = self.bn1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool3(x)

        x = self.dropout2(x)
        x = self.flatten(x)
        x = self.bn2(x)
        
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)  # Использование log_softmax для последнего слоя при классификации


# Определение устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Инициализация модели и перенос на устройство
model = CustomCNN().to(device)

# Проверка модели
print(model)

# Печать информации выбранном устройстве, версии Pytorch  и колисчестве доступных GPU
print("Using device:", device)
print('Pytorch version:', torch.__version__)
print("Num GPUs Available: ", torch.cuda.device_count())

CustomCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.3, inplace=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.4, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (bn2): BatchNorm1d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1152, out_features=512, bias=True)
  (dropout3): Dropout(p=0.25, inplace=False)
  (fc2)

#### Настройка скорости обучения в оптимизаторе Адам для большей точности

In [7]:
# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=0.0007, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, amsgrad=False)
# optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Функция потерь и метрика
criterion = nn.CrossEntropyLoss()

In [8]:
# Перемещение тензоров в память CPU/GPU
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)

#### Функция тренировка модели

In [9]:
def train_model(model, train_loader, val_loader, epochs, device):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total = 0
        correct = 0

        # Тренировочный цикл с tqdm
        train_loop = tqdm(train_loader, total=len(train_loader), leave=True, position=0)
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_loop.set_description(f'Epoch {epoch+1}/{epochs}')
            train_loop.set_postfix(loss=loss.item(), accuracy=100. * correct / total)

        # Валидационный цикл с tqdm
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_loop = tqdm(val_loader, total=len(val_loader), leave=False, position=0)
        with torch.no_grad():
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                _, val_predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (val_predicted == labels).sum().item()

                val_loop.set_postfix(val_loss=val_loss / val_total, val_accuracy=100. * val_correct / val_total)

        print(f'End of Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, Train Accuracy: {100 * correct / total}%, Val Loss: {val_loss / len(val_loader)}, Val Accuracy: {100 * val_correct / val_total}%')


### Обучение модели

In [10]:
# Обучение модели
train_model(model, train_loader, val_loader, 100, device)

Epoch 1/100: 100%|██████████| 352/352 [00:04<00:00, 72.53it/s, accuracy=80.8, loss=0.349]


End of Epoch 1, Train Loss: 0.5197266219183803, Train Accuracy: 80.79555555555555%, Val Loss: 0.3325782457398156, Val Accuracy: 87.65333333333334%


Epoch 2/100: 100%|██████████| 352/352 [00:03<00:00, 88.48it/s, accuracy=86.5, loss=0.236]


End of Epoch 2, Train Loss: 0.36182674956084654, Train Accuracy: 86.48444444444445%, Val Loss: 0.28817116014533123, Val Accuracy: 89.3%


Epoch 3/100: 100%|██████████| 352/352 [00:04<00:00, 86.80it/s, accuracy=88.1, loss=0.221]


...

End of Epoch 98, Train Loss: 0.06877456490871157, Train Accuracy: 97.45777777777778%, Val Loss: 0.2216630271809586, Val Accuracy: 93.52%


Epoch 99/100: 100%|██████████| 352/352 [00:03<00:00, 93.52it/s, accuracy=97.4, loss=0.0198]


End of Epoch 99, Train Loss: 0.07245937548842366, Train Accuracy: 97.4%, Val Loss: 0.23435143450811757, Val Accuracy: 93.57333333333334%


Epoch 100/100: 100%|██████████| 352/352 [00:03<00:00, 93.00it/s, accuracy=97.4, loss=0.0535]
                                                                                       

End of Epoch 100, Train Loss: 0.06981351239664946, Train Accuracy: 97.42888888888889%, Val Loss: 0.23430060052265556, Val Accuracy: 93.52%


In [16]:
# Второй запуск обучения
train_model(model, train_loader, val_loader, 100, device)

Epoch 1/100: 100%|██████████| 352/352 [00:03<00:00, 91.03it/s, accuracy=97.3, loss=0.0997] 


End of Epoch 1, Train Loss: 0.07247184882130982, Train Accuracy: 97.32888888888888%, Val Loss: 0.22549595560689092, Val Accuracy: 93.62666666666667%


Epoch 2/100: 100%|██████████| 352/352 [00:03<00:00, 92.31it/s, accuracy=97.4, loss=0.00857]


End of Epoch 2, Train Loss: 0.07077343759539706, Train Accuracy: 97.40666666666667%, Val Loss: 0.23488176478293993, Val Accuracy: 93.83333333333333%


Epoch 3/100: 100%|██████████| 352/352 [00:03<00:00, 91.60it/s, accuracy=97.2, loss=0.0224]


...

End of Epoch 98, Train Loss: 0.043520418662493204, Train Accuracy: 98.43111111111111%, Val Loss: 0.2756646023273973, Val Accuracy: 93.70666666666666%


Epoch 99/100: 100%|██████████| 352/352 [00:03<00:00, 93.38it/s, accuracy=98.4, loss=0.00852]


End of Epoch 99, Train Loss: 0.045702572403322185, Train Accuracy: 98.40222222222222%, Val Loss: 0.26009413366348055, Val Accuracy: 93.68%


Epoch 100/100: 100%|██████████| 352/352 [00:03<00:00, 92.85it/s, accuracy=98.4, loss=0.00715]
                                                                                       

End of Epoch 100, Train Loss: 0.04604160064826084, Train Accuracy: 98.35111111111111%, Val Loss: 0.26432343184064, Val Accuracy: 93.82666666666667%


In [19]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 10, device)

Epoch 1/10: 100%|██████████| 352/352 [00:03<00:00, 91.59it/s, accuracy=98.6, loss=0.0444] 


End of Epoch 1, Train Loss: 0.03802050384150458, Train Accuracy: 98.6288888888889%, Val Loss: 0.2714279422662773, Val Accuracy: 93.92%


Epoch 2/10: 100%|██████████| 352/352 [00:03<00:00, 92.77it/s, accuracy=98.7, loss=0.000963]


End of Epoch 2, Train Loss: 0.03632934109985151, Train Accuracy: 98.72666666666667%, Val Loss: 0.27596264673491655, Val Accuracy: 93.95333333333333%


Epoch 3/10: 100%|██████████| 352/352 [00:03<00:00, 92.23it/s, accuracy=98.9, loss=0.0399]  


...

End of Epoch 8, Train Loss: 0.026123143968024207, Train Accuracy: 99.06%, Val Loss: 0.2938158785987456, Val Accuracy: 94.1%


Epoch 9/10: 100%|██████████| 352/352 [00:03<00:00, 92.88it/s, accuracy=99.1, loss=0.0683] 


End of Epoch 9, Train Loss: 0.02480715375449191, Train Accuracy: 99.13111111111111%, Val Loss: 0.2920892788255114, Val Accuracy: 94.03333333333333%


Epoch 10/10: 100%|██████████| 352/352 [00:03<00:00, 93.12it/s, accuracy=99.1, loss=0.00636] 
                                                                                       

End of Epoch 10, Train Loss: 0.026033850337129974, Train Accuracy: 99.11111111111111%, Val Loss: 0.2953611172861972, Val Accuracy: 94.18666666666667%


In [21]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.9999), eps=1e-09)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 10, device)

Epoch 1/10: 100%|██████████| 352/352 [00:03<00:00, 92.83it/s, accuracy=99, loss=0.0159]   


End of Epoch 1, Train Loss: 0.026916950249869842, Train Accuracy: 99.03555555555556%, Val Loss: 0.2934353596262507, Val Accuracy: 94.1%


Epoch 2/10: 100%|██████████| 352/352 [00:03<00:00, 90.96it/s, accuracy=99.1, loss=0.136]   


End of Epoch 2, Train Loss: 0.025333497354040446, Train Accuracy: 99.11111111111111%, Val Loss: 0.29565490015103657, Val Accuracy: 94.08666666666667%


Epoch 3/10: 100%|██████████| 352/352 [00:03<00:00, 92.16it/s, accuracy=99.1, loss=0.00194] 


...

End of Epoch 8, Train Loss: 0.027847121663977636, Train Accuracy: 99.05333333333333%, Val Loss: 0.2945322478032213, Val Accuracy: 94.08%


Epoch 9/10: 100%|██████████| 352/352 [00:03<00:00, 92.94it/s, accuracy=99, loss=0.0263]   


End of Epoch 9, Train Loss: 0.02770159761414768, Train Accuracy: 99.04444444444445%, Val Loss: 0.29405296761226857, Val Accuracy: 94.12%


Epoch 10/10: 100%|██████████| 352/352 [00:03<00:00, 92.78it/s, accuracy=99.1, loss=0.0272] 


End of Epoch 10, Train Loss: 0.025586966588393392, Train Accuracy: 99.11555555555556%, Val Loss: 0.2906034413125303, Val Accuracy: 94.1%


In [24]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.9999), eps=1e-09)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 30, device)

Epoch 1/30: 100%|██████████| 352/352 [00:03<00:00, 93.06it/s, accuracy=99.1, loss=0.0204] 


End of Epoch 1, Train Loss: 0.025715921541466905, Train Accuracy: 99.08444444444444%, Val Loss: 0.2898115144916258, Val Accuracy: 94.12666666666667%


Epoch 2/30: 100%|██████████| 352/352 [00:03<00:00, 91.91it/s, accuracy=99.1, loss=0.0978]  


End of Epoch 2, Train Loss: 0.025791819005462457, Train Accuracy: 99.0911111111111%, Val Loss: 0.29513274908255216, Val Accuracy: 94.07333333333334%


Epoch 3/30: 100%|██████████| 352/352 [00:03<00:00, 92.82it/s, accuracy=99.2, loss=0.0176] 


...

End of Epoch 28, Train Loss: 0.023051631353816694, Train Accuracy: 99.19333333333333%, Val Loss: 0.2954013926782093, Val Accuracy: 94.12666666666667%


Epoch 29/30: 100%|██████████| 352/352 [00:03<00:00, 91.82it/s, accuracy=99.2, loss=0.00302] 


End of Epoch 29, Train Loss: 0.024455836951098965, Train Accuracy: 99.18222222222222%, Val Loss: 0.29240329727783043, Val Accuracy: 94.11333333333333%


Epoch 30/30: 100%|██████████| 352/352 [00:03<00:00, 91.84it/s, accuracy=99.2, loss=0.00416] 
                                                                                       

End of Epoch 30, Train Loss: 0.024057218085710538, Train Accuracy: 99.1511111111111%, Val Loss: 0.29354179453231016, Val Accuracy: 94.13333333333334%


#### Функция оценки модели. Промежуточная оценка натренированной модели

In [26]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()  # Переключить модель в режим оценки
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Отключение градиентов для оценки
        for images, labels in tqdm(test_loader, total=len(test_loader), leave=True, position=0, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()  # Суммирование потерь
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Создание тестового DataLoader
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Определение функции потерь
criterion = nn.CrossEntropyLoss()

# Вызов функции оценки модели
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, device)

# Вывод результатов
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Testing: 100%|██████████| 313/313 [00:00<00:00, 502.87it/s]

Test loss: 0.2713372809362619
Test accuracy: 94.19


In [28]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.000001, betas=(0.95, 0.9999), eps=1e-10)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 30, device)

Epoch 1/30: 100%|██████████| 352/352 [00:03<00:00, 92.52it/s, accuracy=99.2, loss=0.041]  


End of Epoch 1, Train Loss: 0.02423134986383957, Train Accuracy: 99.16888888888889%, Val Loss: 0.2919871429675969, Val Accuracy: 94.10666666666667%


Epoch 2/30: 100%|██████████| 352/352 [00:03<00:00, 91.39it/s, accuracy=99.2, loss=0.0383] 


End of Epoch 2, Train Loss: 0.025140236794455137, Train Accuracy: 99.17111111111112%, Val Loss: 0.29465924896362977, Val Accuracy: 94.08%


Epoch 3/30: 100%|██████████| 352/352 [00:03<00:00, 90.69it/s, accuracy=99.2, loss=0.029]   


...

End of Epoch 28, Train Loss: 0.02367223579097878, Train Accuracy: 99.18%, Val Loss: 0.2965739113673315, Val Accuracy: 94.15333333333334%


Epoch 29/30: 100%|██████████| 352/352 [00:03<00:00, 92.18it/s, accuracy=99.2, loss=0.00272] 


End of Epoch 29, Train Loss: 0.02365823408829535, Train Accuracy: 99.2%, Val Loss: 0.29145054151382993, Val Accuracy: 94.14%


Epoch 30/30: 100%|██████████| 352/352 [00:03<00:00, 91.17it/s, accuracy=99.1, loss=0.0133]  
                                                                                       

End of Epoch 30, Train Loss: 0.024654356708858606, Train Accuracy: 99.13333333333334%, Val Loss: 0.2946416636044949, Val Accuracy: 94.12%


#### Промежуточная оценка натренированной модели. 
лучший Test accuracy: 94.19

In [29]:
# Вызов функции оценки модели
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, device)

# Вывод результатов
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Testing: 100%|██████████| 313/313 [00:00<00:00, 492.60it/s]

Test loss: 0.2739715861577124
Test accuracy: 94.19


In [30]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.00001, betas=(0.99, 0.9999), eps=1e-10)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 30, device)

Epoch 1/30: 100%|██████████| 352/352 [00:03<00:00, 93.66it/s, accuracy=99.1, loss=0.00237] 


End of Epoch 1, Train Loss: 0.023428398187620172, Train Accuracy: 99.14888888888889%, Val Loss: 0.29482979224836925, Val Accuracy: 94.14666666666666%


Epoch 2/30: 100%|██████████| 352/352 [00:03<00:00, 92.05it/s, accuracy=99.2, loss=0.0113] 


End of Epoch 2, Train Loss: 0.024218660260389283, Train Accuracy: 99.17333333333333%, Val Loss: 0.2986540622156808, Val Accuracy: 94.11333333333333%


Epoch 3/30: 100%|██████████| 352/352 [00:03<00:00, 91.75it/s, accuracy=99.2, loss=0.0299]  


...

End of Epoch 28, Train Loss: 0.025017651915484086, Train Accuracy: 99.14666666666666%, Val Loss: 0.30403658217292723, Val Accuracy: 94.2%


Epoch 29/30: 100%|██████████| 352/352 [00:03<00:00, 92.76it/s, accuracy=99.2, loss=0.0177]  


End of Epoch 29, Train Loss: 0.02257782665004594, Train Accuracy: 99.20444444444445%, Val Loss: 0.29880153242561774, Val Accuracy: 94.14%


Epoch 30/30: 100%|██████████| 352/352 [00:03<00:00, 91.76it/s, accuracy=99.2, loss=0.0344] 


End of Epoch 30, Train Loss: 0.022285749757925674, Train Accuracy: 99.20444444444445%, Val Loss: 0.2988773182318625, Val Accuracy: 94.13333333333334%


#### Промежуточная оценка натренированной модели

In [31]:
# Вызов функции оценки модели
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, device)

# Вывод результатов
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Testing: 100%|██████████| 313/313 [00:00<00:00, 500.47it/s]

Test loss: 0.275110139952616
Test accuracy: 94.16


In [32]:
# Изменяем скорость обучения, amsgrad, weight_decay
optimizer = optim.Adam(model.parameters(), lr=0.00001, betas=(0.99, 0.9999), eps=1e-10)  

# N-запуск обучения на дополнительные эпохи
train_model(model, train_loader, val_loader, 100, device)

Epoch 1/100: 100%|██████████| 352/352 [00:03<00:00, 90.88it/s, accuracy=99.2, loss=0.00785] 


End of Epoch 1, Train Loss: 0.02296004800882656, Train Accuracy: 99.2%, Val Loss: 0.29655594891726467, Val Accuracy: 94.16%


Epoch 2/100: 100%|██████████| 352/352 [00:03<00:00, 91.97it/s, accuracy=99.2, loss=0.00298] 


End of Epoch 2, Train Loss: 0.021915379305772312, Train Accuracy: 99.24444444444444%, Val Loss: 0.2949221867720707, Val Accuracy: 94.12666666666667%


Epoch 3/100: 100%|██████████| 352/352 [00:03<00:00, 93.54it/s, accuracy=99.3, loss=0.0403]  


...

End of Epoch 98, Train Loss: 0.020233295979943996, Train Accuracy: 99.28%, Val Loss: 0.3524255714521317, Val Accuracy: 94.06666666666666%


Epoch 99/100: 100%|██████████| 352/352 [00:03<00:00, 91.44it/s, accuracy=99.2, loss=0.069]   


End of Epoch 99, Train Loss: 0.020603235125814866, Train Accuracy: 99.24888888888889%, Val Loss: 0.3630552977260392, Val Accuracy: 94.10666666666667%


Epoch 100/100: 100%|██████████| 352/352 [00:03<00:00, 93.02it/s, accuracy=99.3, loss=0.00147] 
                                                                                       

End of Epoch 100, Train Loss: 0.021489021458943502, Train Accuracy: 99.26222222222222%, Val Loss: 0.3188973629720888, Val Accuracy: 94.14666666666666%


#### Конечная оценка натренированной модели

In [33]:
# Вызов функции оценки модели
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, device)

# Вывод результатов
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Testing: 100%|██████████| 313/313 [00:00<00:00, 505.88it/s]

Test loss: 0.278210402267744
Test accuracy: 94.17


### Подготовка сабмиссии
После обучения модели  используем тестовый набор данных для создания предсказаний.  
Записываем файл для сабмиссии на Kaggle.

In [27]:
# Предсказание на тестовом наборе данных
test_images = torch.tensor(test_data.drop('label', axis=1).values).float().reshape(-1, 1, 28, 28) / 255.0
test_images = test_images.to(device)
model.eval()
with torch.no_grad():
    outputs = model(test_images)
    predicted_labels = outputs.argmax(dim=1).cpu().numpy()

# Создание DataFrame для сабмиссии
submission = pd.DataFrame({
    'Id': test_data.index,
    'Category': predicted_labels
})

# Сохранение файла сабмиссии
submission.to_csv('submission.csv', index=False)